# The Disappearing Embedding Function

Previously, to use vector databases, you had to do the embedding process yourself and interact with the system using vectors directly.
With this new release of LanceDB, we make it much more convenient so you don't need to worry about that at all.

1. We present you with sentence-transformer, openai, and openclip embedding functions that can be saved directly as table metadata
2. You no longer have to generate the vectors directly either during query time or ingestion time
3. The embedding function interface is extensible so you can create your own
4. The function is persisted as table metadata so you can use it across sessions

In [1]:
import lancedb

## Multi-modal search made easy

In this example we'll multi-modal image search using:
- Oxford Pet dataset
- OpenClip model
- LanceDB

First, download the dataset from https://www.robots.ox.ac.uk/~vgg/data/pets/

### Define embedding function

We'll use the OpenClipEmbeddingFunction here for multi-modal image search.

In [ ]:
from lancedb.embeddings import EmbeddingFunctionRegistry

registry = EmbeddingFunctionRegistry.get_instance()
clip = registry.get("openclip").create()

### Declare the table schema

pydantic makes it really easy to bridge python and our database schema.

We'll declare a new model that subclasses LanceModel to represent the table.
This table has two columns, corresponding to the ones defined above in the embedding function.
The embedding function defines the number of dimensions in its vectors so you don't need to 
look it up.

Here because we're working with images, we add a convenience method to open the image and return a PIL Image so it can be visualized in Jupyter Notebook

In [ ]:
from PIL import Image
from lancedb.pydantic import LanceModel, Vector

class Pets(LanceModel):
    vector: Vector(clip.ndims) = clip.VectorField()
    image_uri: str = clip.SourceField()
    
    @property
    def image(self):
        return Image.open(self.image_uri)    

### Create the table

First we connect to a local lancedb directory

In [ ]:
db = lancedb.connect("~/.lancedb")

Next we get all of the paths for the images we downloaded and create a table.
Notice that we didn't have to worry about generating the image embeddings ourselves.

In [ ]:
import pandas as pd
from pathlib import Path
from random import sample

if "pets" in db:
    table = db["pets"]
else:
    table = db.create_table("pets", schema=Pets)
    # use a sampling of 1000 images
    p = Path("~/Downloads/images").expanduser()
    uris = [str(f) for f in p.iterdir()]
    uris = sample(uris, 1000)
    table.add(pd.DataFrame({"image_uri": uris}))

### Querying via text

We also don't need to generate the embeddings when querying either.
LanceDB does that automatically so you can query directly using text input.

The pydantic model we declared for the table schema also makes it really easy for us to work with the search results

In [ ]:
rs = table.search("dog").limit(3).to_pydantic(Pets)
rs[0].image

### Querying via images

The great thing about CLIP is that it's multi-modal.
So you can search using not just text but images as well.

Create a query image using PIL

In [ ]:
from PIL import Image
p = Path("/Users/changshe/Downloads/images/samoyed_100.jpg")
query_image = Image.open(p)
query_image

Pass in the query_image to the search API

In [ ]:
rs = table.search(query_image).limit(3).to_pydantic(Pets)
rs[0].image

### Persistence

Embedding functions are persisted as table metadata so it's much easier to use across sessions.

For example we can recreate the database connection and table object

In [ ]:
db = lancedb.connect("~/.lancedb")
table = db["pets"]

We can observe that it's read out as table metadata

In [ ]:
import json

json.loads(table.schema.metadata[b"embedding_functions"])[0]

And we can also run queries as before without having to reinstantiate the embedding function explicitly

In [ ]:
rs = table.search("big dog").limit(3).to_pydantic(Pets)
rs[0].image

## LanceDB makes multimodal AI easy



- LanceDB's new embedding functions feature makes it easy for builders of LLM apps
- You no longer need to manually encode the data yourself
- You no longer need to figure out how many dimensions is your vector
- You no longer need to manually encode the query
- And with the right embedding model, you can search way more than just text